# Email category prediction

In [1]:
# import supporting functions from python file

import supporting_modules.supp_functions as sfn

In [2]:
# import dataset

from sklearn.datasets import fetch_20newsgroups

### Fetch the training data and the test ones

In [4]:
train_data = fetch_20newsgroups(subset="train")
test_data = fetch_20newsgroups(subset="test")